## 'Smart' Order Management Simulation Software
1. **Event-Driven Architecture**: Processes orders in true time sequence
2. **Dynamic Worker Management**: Handles attendant availability changes
3. **Intelligent Routing**: Maximizes order acceptance while respecting constraints

In [40]:
import heapq
import random
from dataclasses import dataclass, field
from typing import List, Dict

## 1. Event Class
The core building block for managing temporal events

In [41]:
@dataclass(order=True)
class Event:
    time: float
    type: str = field(compare=False)  # states include: "order_arrival" or "order_completion"
    data: dict = field(compare=False)

## 2. Attendant Class
Models each worker with state tracking and constraint enforcement

In [42]:
class Attendant:
    def __init__(self, name: str):
        self.name = name
        self.active_orders: List[float] = []  # Start times of active orders
        self.status: str = "active"  # statuss i.e 'active', 'inactive', 'overloaded'
        self.next_available: float = 0  # Next time slot for new orders

    def update_state(self, current_time: float):
        """Update order completion status and availability"""
        # Remove completed orders i.e, orders older than 5 secs
        self.active_orders = [t for t in self.active_orders if t + 5 > current_time]

        # Calculate next availability
        if len(self.active_orders) == 0:
            self.next_available = current_time
        elif len(self.active_orders) == 1:
            self.next_available = max(current_time, self.active_orders[0] + 3)
        else:
            self.next_available = max(
                self.active_orders[0] + 5,  # First order completes
                self.active_orders[1] + 3  # Cooldown from second order
            )

    def can_accept(self, current_time: float) -> bool:
        if self.status != "active":
            return False
        return current_time >= self.next_available and len(self.active_orders) < 2

    def start_order(self, start_time: float):
        self.active_orders.append(start_time)
        self.active_orders.sort()


## 3. Orchestrator System
**core controller we use for managing the entire order processing flow**

### Key Responsibilities:
1. **Event Management**:
   - Maintains priority queue of future events (orders/completions)
   - Processes events in chronological order
   - Auto-schedules order completion events

2. **Intelligent Routing**:
   - Evaluates all attendant's availability in realtime
   - Selects optimal attendant using:
     - Current workload (number of active orders)
     - Random factor for load balancing
     - Status awareness (active/inactive)

3. **State Synchronization**:
   - updates all attendant;s availability before each decision
   - Handles attendant status changes (active/inactive)
   - Maintains global simulation clock

4. **Failure Handling**:
   - Automatically rejects orders when no attendants available
   - Tracks processed/rejected orders statistics and logs statuses to stdout

In [43]:
class Orchestrator:
    def __init__(self, attendants: List[Attendant]):
        self.attendants = {a.name: a for a in attendants}
        self.event_queue = []
        self.current_time = 0.0
        self.processed = 0
        self.rejected = 0

    def add_order(self, arrival_time: float):
        heapq.heappush(self.event_queue, Event(arrival_time, 'order_arrival', {}))

    def update_attendant_status(self, name: str, status: str):
        """Handle attendant shutdown/revival events"""
        self.attendants[name].status = status
        if status == 'active':
            self.attendants[name].update_state(self.current_time)

    def process_events(self):
        while self.event_queue:
            event = heapq.heappop(self.event_queue)
            self.current_time = event.time

            if event.type == 'order_arrival':
                self.handle_order_arrival()
            elif event.type == 'order_completion':
                self.handle_order_completion(event.data['attendant'])

    def handle_order_arrival(self):
        candidates = []
        for name, attendant in self.attendants.items():
            attendant.update_state(self.current_time)

            if attendant.can_accept(self.current_time):
                # calculate priority, i.e, we prefer attendants with fewer active orders
                priority = (len(attendant.active_orders), random.random())
                candidates.append((priority, name))

        if candidates:
            # Select attendant with best availability
            _, best = min(candidates)
            self.attendants[best].start_order(self.current_time)
            self.processed += 1

            # Schedule order completion
            heapq.heappush(
                self.event_queue,
                Event(self.current_time + 5, 'order_completion', {'attendant': best})
            )

            print(f"[{self.current_time:.2f}] Order accepted by {best}")
        else:
            self.rejected += 1
            print(f"[{self.current_time:.2f}] Order rejected")

    def handle_order_completion(self, attendant_name: str):
        print(f"[{self.current_time:.2f}] Order completed by {attendant_name}")
        self.attendants[attendant_name].update_state(self.current_time)


## 4. Example Usage
Demonstrates the system in action
We can interrupt execution by add

In [44]:
if __name__ == "__main__":
    # we Initialize system with 2 attendants
    attendants = [Attendant("Alice"), Attendant("Bob")]
    orchestrator = Orchestrator(attendants)

    # Simulate random order arrivals
    for _ in range(20):
        orchestrator.add_order(random.uniform(0, 30))

    # Simulate attendant B going offline at time 10
    orchestrator.add_order(10.0)  # Special event for shutdown
    heapq.heappush(orchestrator.event_queue,
                   Event(10.0, 'order_completion', {'attendant': 'Bob'}))

    orchestrator.process_events()

    print(f"\nTotal processed: {orchestrator.processed}")
    print(f"Total rejected: {orchestrator.rejected}")

[5.67] Order accepted by Alice
[6.09] Order accepted by Bob
[7.82] Order rejected
[9.46] Order accepted by Alice
[10.00] Order completed by Bob
[10.00] Order accepted by Bob
[10.35] Order rejected
[10.67] Order completed by Alice
[10.93] Order rejected
[11.09] Order completed by Bob
[12.25] Order rejected
[12.35] Order rejected
[12.99] Order accepted by Alice
[14.46] Order completed by Alice
[14.46] Order accepted by Bob
[15.00] Order completed by Bob
[15.31] Order rejected
[16.71] Order accepted by Alice
[17.14] Order rejected
[17.99] Order completed by Alice
[19.39] Order accepted by Bob
[19.46] Order completed by Bob
[20.77] Order accepted by Alice
[21.36] Order rejected
[21.71] Order completed by Alice
[21.76] Order rejected
[24.39] Order completed by Bob
[25.49] Order accepted by Bob
[25.77] Order completed by Alice
[29.20] Order accepted by Alice
[29.77] Order accepted by Bob
[30.49] Order completed by Bob
[34.20] Order completed by Alice
[34.77] Order completed by Bob

Total pro